In [1]:
# SVM versus Neural Networks

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
import pandas as pd
import tensorflow as tf



2023-01-27 13:31:01.535519: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
tele_df = pd.read_csv('bank_telemarketing.csv')
tele_df.head()


,Age,Job,Marital_Status,Education,Default_Credit,Housing_Loan,Personal_Loan,Subscribed
0,56,other,married,Primary_Education,no,no,no,no
1,37,services,married,Secondary_Education,no,yes,no,no
2,40,admin,married,Primary_Education,no,no,no,no
3,56,services,married,Secondary_Education,no,no,yes,no
4,59,admin,married,Professional_Education,no,no,no,no


In [3]:
# Generate categorical variables list
tele_cat = tele_df.dtypes[tele_df.dtypes == 'object'].index.tolist()
print(tele_cat)

# Check the # of unique values in each column
tele_df[tele_cat].nunique()

['Job', 'Marital_Status', 'Education', 'Default_Credit', 'Housing_Loan', 'Personal_Loan', 'Subscribed']


Job               9
Marital_Status    3
Education         4
Default_Credit    2
Housing_Loan      2
Personal_Loan     2
Subscribed        2
dtype: int64

In [6]:
# Create OneHotEncoder instance
enc = OneHotEncoder(sparse = False)

# Fit and transform the encode using the categorical var list
encode_df = pd.DataFrame(enc.fit_transform(tele_df[tele_cat]))

# Add the encoded var names to the DF
encode_df.columns = enc.get_feature_names(tele_cat)
encode_df.head()

/opt/miniconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,Job_admin,Job_blue-collar,Job_entrepreneur,Job_management,Job_other,Job_retired,Job_self-employed,Job_services,Job_technician,Marital_Status_divorced,...,Education_Secondary_Education,Education_Tertiary_Education,Default_Credit_no,Default_Credit_yes,Housing_Loan_no,Housing_Loan_yes,Personal_Loan_no,Personal_Loan_yes,Subscribed_no,Subscribed_yes
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0


In [7]:
# Merge one-hot encoded features and drop the originals
tele_df = tele_df.merge(encode_df, left_index = True, right_index = True)
tele_df = tele_df.drop(tele_cat, 1)
tele_df.head()

/var/folders/f3/5zcd9_1j7d3546ws7g9t5fw00000gp/T/ipykernel_94017/1150903973.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  tele_df = tele_df.drop(tele_cat, 1)


,Age,Job_admin,Job_blue-collar,Job_entrepreneur,Job_management,Job_other,Job_retired,Job_self-employed,Job_services,Job_technician,...,Education_Secondary_Education,Education_Tertiary_Education,Default_Credit_no,Default_Credit_yes,Housing_Loan_no,Housing_Loan_yes,Personal_Loan_no,Personal_Loan_yes,Subscribed_no,Subscribed_yes
0,56,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
1,37,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
2,40,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
3,56,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
4,59,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0


In [8]:
# Remove target (loan status) from features
y = tele_df.Subscribed_yes.values
X = tele_df.drop(columns = ['Subscribed_no', 'Subscribed_yes']).values

# Split training and test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, stratify = y)

# Scaler instance
scaler = StandardScaler()

# Fit the scaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


In [9]:
# TRAINING AND EVALUATING SVM
# Create the SVM model
svm = SVC(kernel = 'linear')

# Train the model
svm.fit(X_train, y_train)

# Evaluate
y_pred = svm.predict(X_test_scaled)
print(f'SVM model accuracy: {accuracy_score(y_test, y_pred):.3f}')

SVM model accuracy: 0.873


In [10]:
# NEURAL NETWORK

# Define the model 
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 10
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units = hidden_nodes_layer1, input_dim = number_input_features, activation = 'relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units = hidden_nodes_layer2, activation = 'relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

# Compile the model and customize metrics
nn.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
       
       
       

2023-01-27 13:47:47.092600: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [11]:
# train the model

fit_model = nn.fit(X_train_scaled, y_train, epochs = 50)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose = 2)

Epoch 1/50
715/715 [==============================] - 4s 2ms/step - loss: 0.4002 - accuracy: 0.8670
Epoch 2/50
715/715 [==============================] - 1s 2ms/step - loss: 0.3719 - accuracy: 0.8735
Epoch 3/50
715/715 [==============================] - 1s 2ms/step - loss: 0.3691 - accuracy: 0.8735
Epoch 4/50
715/715 [==============================] - 1s 2ms/step - loss: 0.3680 - accuracy: 0.8734
Epoch 5/50
715/715 [==============================] - 1s 2ms/step - loss: 0.3670 - accuracy: 0.8734
Epoch 6/50
715/715 [==============================] - 1s 2ms/step - loss: 0.3662 - accuracy: 0.8732
Epoch 7/50
715/715 [==============================] - 1s 2ms/step - loss: 0.3659 - accuracy: 0.8733
Epoch 8/50
715/715 [==============================] - 1s 2ms/step - loss: 0.3652 - accuracy: 0.8736
Epoch 9/50
715/715 [==============================] - 1s 2ms/step - loss: 0.3647 - accuracy: 0.8736
Epoch 10/50
715/715 [==============================] - 1s 2ms/step - loss: 0.3648 - accuracy: 0.8735

In [12]:
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Loss: 0.36841586232185364, Accuracy: 0.872309684753418
